# 第4章　scikit-learn を用いたトランスクリプトームデータの分類

- 澤田 高志
- 清水 秀幸

##### 入力4-1

In [ ]:
!pip install optuna

##### 入力4-2

In [ ]:
# パッケージのインポート
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import sklearn

##### 入力4-3

In [ ]:
# バージョンの確認
print('numpy: ', np.__version__)
print('pandas: ', pd.__version__)
print('matplotlib: ', matplotlib.__version__)
print('sklearn: ', sklearn.__version__)
print('seaborn: ', sns.__version__)
print('optuna: ', optuna.__version__)

##### 入力4-4

In [ ]:
# アップロードしたcsvファイルを読み込む
gse_mRNA_exprs_normal_selected = pd.read_csv(
    '/content/GSE36376_normal.csv', index_col=0
)
gse_mRNA_exprs_tumor_selected = pd.read_csv(
    '/content/GSE36376_tumor.csv', index_col=0
)

##### 入力4-5

In [ ]:
# ようやく前処理が終わり，ここから肝臓がんと周辺正常組織の区別を行うための
# 分類器を構築することにしよう

# 新たにclass labelを作り，正常を0，がんを1とする
gse_mRNA_exprs_normal_selected.loc['class'] = 0
gse_mRNA_exprs_tumor_selected.loc['class'] = 1

##### 入力4-6

In [ ]:
# 簡単な分類器を作るために，データを訓練データとテストデータに分割する.
# これは，scikit-learnのtrain_test_split関数を使うことで実現できる
# デフォルトでは順番もシャッフルされている
from sklearn.model_selection import train_test_split

# 転置したものをtrain_test_splitで分ける.train_size=0.75で元のデータを75%と25%に分けられる
gse_mRNA_exprs_normal_selected_train, gse_mRNA_exprs_normal_selected_test = train_test_split(
    gse_mRNA_exprs_normal_selected.T, train_size=0.75, random_state=0)
gse_mRNA_exprs_tumor_selected_train, gse_mRNA_exprs_tumor_selected_test = train_test_split(
    gse_mRNA_exprs_tumor_selected.T, train_size=0.75, random_state=0)

##### 入力4-7

In [ ]:
# X_train, y_train, X_test, y_testデータを作り出す
gse_mRNA_exprs_train = pd.concat(
    [gse_mRNA_exprs_normal_selected_train, gse_mRNA_exprs_tumor_selected_train]
)
# X_trainは発現量のデータ
X_train = gse_mRNA_exprs_train.iloc[:, 0:-1]
# y_trainはそのサンプルが0(正常)か1(がん)かのデータ. -1で最後の列を指定できる.
y_train = gse_mRNA_exprs_train.iloc[:, -1]
gse_mRNA_exprs_test = pd.concat(
    [gse_mRNA_exprs_normal_selected_test, gse_mRNA_exprs_tumor_selected_test]
)
X_test = gse_mRNA_exprs_test.iloc[:, 0:-1]
y_test = gse_mRNA_exprs_test.iloc[:, -1]

##### 入力4-8

In [ ]:
# データの確認
X_train.describe()

##### 入力4-9

In [ ]:
# 元々quantile正規化を行っていたデータなので，平均も分散も似たような値でまとまっているのだが
# 標準化するほうが一般的なので実行してみよう
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
# StandardScalerのfitメソッドを取ってくる
# 訓練データの平均値と標準偏差の情報がsc.に与えられる.
sc.fit(X_train)

# 訓練データの平均と標準偏差をもとに標準化
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# データフレーム型にして確認
X_train_std_df = pd.DataFrame(X_train_std)
X_train_std_df.columns = X_train.columns
# 浮動小数点数の関係で平均は完全に0になるわけではない. 標準偏差も完全に1ではない
X_train_std_df.describe()

##### 入力4-10

In [ ]:
# データフレーム型にして確認
X_test_std_df = pd.DataFrame(X_test_std)
X_test_std_df.columns = X_test.columns
# テストデータセットは訓練データセットの平均で差し引かれ，訓練データセットの標準偏差で割られている
# そのため，平均値は0に近い値，標準偏差は1に近い値になっている
X_test_std_df.describe()

##### 入力4-11

In [ ]:
# さて，ようやく分類器を扱ってみる
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC

# 線形SVMのインスタンスを生成. ハイパーパラメータはとりあえずC=1, gamma=1
clf = SVC(kernel='linear', C=1, gamma=1, random_state=0, probability=True)
# 線形SVMのモデルにトレーニングデータを適合させる
clf.fit(X_train_std, y_train)

# まずは訓練データでの成果を確認する

# clf.predict()ではラベルを返すが，clf.predict_proba()では確率を返す.
# 例えば，clf.predict_proba(X_train_std)[:, 1]の結果，ラベル1である確率が0.05だったサンプルについて
# clf.predict(X_train_std)の値は0になる
y_train_pred = clf.predict_proba(X_train_std)[:, 1]
roc_auc_score(y_train, y_train_pred)

##### 入力4-12

In [ ]:
# test scoreは？
y_pred = clf.predict_proba(X_test_std)[:, 1]
test_score = roc_auc_score(y_test, y_pred)
print(test_score)

##### 入力4-13

In [ ]:
from sklearn.metrics import auc, roc_curve

# ROC-AUC
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_pred)
auc_test = auc(fpr_test, tpr_test)

# ROC曲線を描く
plt.plot([0, 1], [0, 1], color='black', linestyle='--')

plt.plot(fpr_test, tpr_test, label='SVM (AUC = %.3f)' % auc_test)
plt.fill_between(fpr_test, tpr_test, 0, alpha=0.1)

plt.legend()
plt.title('SVM')
plt.xlabel('False_Positive_Rate')
plt.ylabel('True_Positive_Rate')
plt.grid(True)
plt.show()

##### 入力4-14

In [ ]:
# i番目のy_pred，つまりy_pred[i]が0.5より大きい場合1に，そうでない場合0に変換する.
# リスト内包表記とよばれる記法で，普通のif文と比べ速度が上がる
y_pred_label = [1 if y_pred[i] > 0.5 else 0 for i in range(len(y_pred))]

from sklearn.metrics import classification_report, confusion_matrix

confusion_matrix(y_test, y_pred_label)

##### 入力4-15

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_label))

##### 入力4-16

In [ ]:
from sklearn.model_selection import train_test_split

# 転置したものをtrain_test_splitで分ける
# まずはtrainvalデータセットとtestデータに分ける.
# testは全体の25%，trainvalが75%
gse_mRNA_exprs_normal_selected_trainval, gse_mRNA_exprs_normal_selected_test = train_test_split(
    gse_mRNA_exprs_normal_selected.T, train_size=0.75, random_state=0)
gse_mRNA_exprs_tumor_selected_trainval, gse_mRNA_exprs_tumor_selected_test = train_test_split(
    gse_mRNA_exprs_tumor_selected.T, train_size=0.75, random_state=0)

# 次にtrainvalデータセットをtrainとvalに分ける.
# trainが全体の50%，valが全体の25%
gse_mRNA_exprs_normal_selected_train, gse_mRNA_exprs_normal_selected_val = train_test_split(
    gse_mRNA_exprs_normal_selected_trainval, train_size=0.667, random_state=0)
gse_mRNA_exprs_tumor_selected_train, gse_mRNA_exprs_tumor_selected_val = train_test_split(
    gse_mRNA_exprs_tumor_selected_trainval, train_size=0.667, random_state=0)

##### 入力4-17

In [ ]:
# trainデータ，validationデータ，testデータを作り出す
gse_mRNA_exprs_train = pd.concat(
    [gse_mRNA_exprs_normal_selected_train, gse_mRNA_exprs_tumor_selected_train]
)
X_train = gse_mRNA_exprs_train.iloc[:, 0:-1]
y_train = gse_mRNA_exprs_train.iloc[:, -1]

gse_mRNA_exprs_val = pd.concat(
    [gse_mRNA_exprs_normal_selected_val, gse_mRNA_exprs_tumor_selected_val]
)
X_val = gse_mRNA_exprs_val.iloc[:, 0:-1]
y_val = gse_mRNA_exprs_val.iloc[:, -1]

gse_mRNA_exprs_test = pd.concat(
    [gse_mRNA_exprs_normal_selected_test, gse_mRNA_exprs_tumor_selected_test]
)
X_test = gse_mRNA_exprs_test.iloc[:, 0:-1]
y_test = gse_mRNA_exprs_test.iloc[:, -1]

##### 入力4-18

In [ ]:
# 標準化を行う
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)
X_val_std = sc.transform(X_val)
X_test_std = sc.transform(X_test)

##### 入力4-19

In [ ]:
kernel = ['linear', 'rbf']
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gamma = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

# 当面の最高スコアを0としておいて後で更新していく
best_score = 0

# グリッドサーチの実装のためにfor文を積み重ねる
for i in kernel:
    for j in C:
        for k in gamma:
            clf = SVC(kernel=i, C=j, gamma=k, random_state=0, probability=True)
            clf.fit(X_train_std, y_train)
            y_val_pred = clf.predict_proba(X_val_std)[:, 1]
            gs_score = roc_auc_score(y_val, y_val_pred)

            # ハイパーパラメータやスコアの中身を詳しく見たい場合，以下コードの#を外す
            # print('kernel: {}'.format(i), 'C: {}'.format(j), 'gamma: {}'.format(k))
            # print('score: {}'.format(gs_score))

            # gs_scoreが現在のbest_scoreを超えた場合，best_scoreとbest_paramsを更新する
            if gs_score > best_score:
                best_score = gs_score
                best_params = {'kernel': i, 'C': j, 'gamma': k}

##### 入力4-20

In [ ]:
print(best_score)
print(best_params)

##### 入力4-21

In [ ]:
# これをテストデータセットに適用する.
from sklearn.metrics import roc_auc_score

clf = SVC(**best_params, random_state=0, probability=True)
clf.fit(X_train_std, y_train)
y_pred = clf.predict_proba(X_test_std)[:, 1]
test_score = roc_auc_score(y_test, y_pred)
print(test_score)

##### 入力4-22

In [ ]:
from sklearn.metrics import auc, roc_curve

# ROC-AUC
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_pred)
auc_test = auc(fpr_test, tpr_test)

# ROC曲線を描く
plt.plot([0, 1], [0, 1], color='black', linestyle='--')

plt.plot(fpr_test, tpr_test, label='GridSearch+SVM (AUC = %.3f)' % auc_test)
plt.fill_between(fpr_test, tpr_test, 0, alpha=0.1)

plt.legend()
plt.title('GridSearch+SVM')
plt.xlabel('False_Positive_Rate')
plt.ylabel('True_Positive_Rate')
plt.grid(True)
plt.show()

##### 入力4-23

In [ ]:
# y_predが0.5より大きい場合1に，そうでない場合0にする.
y_pred_label = [1 if y_pred[i] > 0.5 else 0 for i in range(len(y_pred))]

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred_label))
print(classification_report(y_test, y_pred_label))

##### 入力4-24

In [ ]:
# GridSearchCVでより簡単に検証ができる
from sklearn.model_selection import GridSearchCV
hyperparameters_gs = {
    'kernel': ['linear', 'rbf'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'random_state': [0],
    'probability': [True],
}

# K分割交差検証の，「分割」の部分. KFold()ではなく，StratifiedKFold()を使うことで
# NormalとTumorの比率を揃えたまま分割できる
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

grid_search = GridSearchCV(
      # 'roc_auc'も選択できるが，ここではF1スコアを最大とするハイパーパラメータを探す
      SVC(), param_grid=hyperparameters_gs, cv=skf, scoring='f1'
)
grid_search.fit(X_train_std, y_train)

##### 入力4-25

In [ ]:
grid_search.best_score_

##### 入力4-26

In [ ]:
grid_search.best_estimator_

##### 入力4-27

In [ ]:
model = grid_search.best_estimator_
y_pred = model.predict_proba(X_test_std)[:, 1]
score = roc_auc_score(y_test, y_pred)
score

##### 入力4-28

In [ ]:
from sklearn.metrics import auc, roc_curve

# ROC-AUC
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_pred)
auc_test = auc(fpr_test, tpr_test)

# ROC曲線を描く
plt.plot([0, 1], [0, 1], color='black', linestyle='--')

plt.plot(fpr_test, tpr_test, label='GridSearchCV+SVM (AUC = %.3f)' % auc_test)
plt.fill_between(fpr_test, tpr_test, 0, alpha=0.1)

plt.legend()
plt.title('GridSearchCV+SVM')
plt.xlabel('False_Positive_Rate')
plt.ylabel('True_Positive_Rate')
plt.grid(True)
plt.show()

##### 入力4-29

In [ ]:
# y_predが0.5より大きい場合1に，そうでない場合0にする.
y_pred_label = [1 if y_pred[i] > 0.5 else 0 for i in range(len(y_pred))]

from sklearn.metrics import classification_report, confusion_matrix
# 混同行列とclassification_reportを表示する.
print(confusion_matrix(y_test, y_pred_label))
print(classification_report(y_test, y_pred_label))

##### 入力4-30

In [ ]:
from functools import partial
import optuna

from sklearn.model_selection import cross_val_score

def objective_svc(trial):
    '''最適化すべき目的関数'''
    params_svc = {
    # trialはハイパーパラメータの範囲を指定する
    # 'kernel'はカテゴリカル変数: 'linear'か'rbf'かをoptunaに選ばせる
    'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf']),
    # 'C'は連続値だが，対数的な取り方を行う. 0.001から1000までの値をOptunaに選ばせる
    'C': trial.suggest_float('C', 1e-3, 1e3, log=True),
    # 'gamma'も連続値だが，対数的な取り方を行う. 0.001から1000までの値をOptunaに選ばせる
    'gamma': trial.suggest_float('gamma', 1e-3, 1e3, log=True),
    'random_state': 0,
    }
    clf = SVC(**params_svc)

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
    # グリッドサーチとは別の最適化手法を用いるため，GridSearchCVは使わない.
    # 代わりにcross_val_score()で交差検証を行う
    # cross_val_score()の性能指標について，'roc_auc'も選択できるが，ここではF1スコアを用いた
    # return metricで最適化すべき目的関数をmetricに指定する.
    train_scores = cross_val_score(clf, X_train_std, y_train, cv=skf, scoring='f1')
    metric = train_scores.mean()
    return metric

##### 入力4-31

In [ ]:
obj_svc = partial(objective_svc)
# セッションの作成
sampler = optuna.samplers.TPESampler(seed=0)
# F1スコアを最大化するためにdirection='maximize'にする
study_svc = optuna.create_study(sampler=sampler, direction='maximize')
# 回数を指定する.
study_svc.optimize(obj_svc, n_trials=100)

##### 入力4-32

In [ ]:
print(study_svc.best_params)
print(study_svc.best_value)

##### 入力4-33

In [ ]:
study_svc.trials_dataframe()

##### 入力4-34

In [ ]:
# 訓練データのスコア
optuna_params = study_svc.best_params
clf = SVC(**optuna_params, probability=True)
# 線形SVMのモデルにトレーニングデータを適合させる
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
# cross_val_score()の性能指標について、'roc_auc'も選択できるが、ここではF値を用いた
train_scores = cross_val_score(clf, X_train_std, y_train, cv=skf, scoring='f1')
metric = train_scores.mean()
print(metric)

##### 入力4-35

In [ ]:
# test scoreは？
from sklearn import metrics

clf.fit(X_train_std, y_train)
y_pred = clf.predict_proba(X_test_std)[:, 1]
roc_auc_score(y_test, y_pred)

##### 入力4-36

In [ ]:
from sklearn.metrics import auc, roc_curve

# ROC-AUC
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_pred)
auc_test = auc(fpr_test, tpr_test)

# ROC曲線を描く
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.plot(fpr_test, tpr_test, label='Optuna+SVM (AUC = %.3f)' % auc_test)

plt.fill_between(fpr_test, tpr_test, 0, alpha=0.1)
plt.legend()

plt.title('Optuna+SVM')
plt.xlabel('False_Positive_Rate')
plt.ylabel('True_Positive_Rate')
plt.grid(True)
plt.show()

##### 入力4-37

In [ ]:
# y_predが0.5より大きい場合1に，そうでない場合0にする.
y_pred_label = [1 if y_pred[i] > 0.5 else 0 for i in range(len(y_pred))]

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred_label))
print(classification_report(y_test, y_pred_label))